In [1]:
import streamlit as st
import numpy as np
import cv2
from PIL import Image
import tensorflow as tf
import torch
from ultralytics import YOLO

In [4]:
def vid_with_label_1stg(img):

    
    model_path = "models\yolo_custom_model.pt"
    model = YOLO(model_path)
    # img = cv2.resize(img, (720, int(720 * (9 / 16))))
    
    if torch.cuda.is_available():
      res = model.track(img, conf = 0.5, persist = True, device = 'cuda' )
    else:
      res = model.track(img, conf = 0.5, persist = True)
    
    res_plotted = res[0].plot()
    id2label = {
    '0' : 'Anger',
    '1' : 'Happy',
    '2' : 'Panic',
    '3' : 'Sadness'
    }
    # yolo 에서 가져온 값들 따로 처리해보기
    try:
        # start_point , end_point = np.array_split(res[0].boxes.xyxy.cpu().numpy().tolist()[0],2)
        # score = str(round(res[0].boxes.conf.cpu().numpy().tolist()[0]*100,2))+ '%'
   
        label = id2label[str(int(res[0].boxes.cls.cpu().numpy().tolist()[0]))]
        return res_plotted, label
        # results_str = label + ':'+ score
    except Exception as e:
       return res_plotted, None
    
# 비디오 캡처 객체 생성
cap = cv2.VideoCapture(0)

while True:
    
    # 프레임들 읽기
    ret, frame = cap.read()

    img, label = vid_with_label_1stg(frame)
    
    if label is not None:
        print(label)

    # 영상 출력
    
    cv2.imshow('Real-time Video', img)

    # 'q' 키를 누르면 종
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 객체 해제
cap.release()
cv2.destroyAllWindows()


0: 480x640 (no detections), 9.5ms
Speed: 3.0ms preprocess, 9.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 6.6ms
Speed: 1.4ms preprocess, 6.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 5.1ms
Speed: 1.0ms preprocess, 5.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 6.0ms
Speed: 1.5ms preprocess, 6.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 7.1ms
Speed: 1.0ms preprocess, 7.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 9.5ms
Speed: 1.0ms preprocess, 9.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 6.5ms
Speed: 1.0ms preprocess, 6.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 9.5ms
Speed: 1.0ms preprocess, 9.5ms inference, 0.0ms 